<a href="https://colab.research.google.com/github/krishnapratapsingh788-eng/python-project/blob/main/Mini%20chatbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.0 MB/s eta 0:00:00


In [ ]:
pip install googlesearch-python

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("37ZTrHIMQQ8lARjOOne9Q7k0UeE_ULFMVdoMX9emuqnpqKxk")

In [ ]:
%%writefile app.py
import streamlit as st
import json, random, os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from googlesearch import search

# ✅ Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

# ---------------- USER DATA PERSISTENCE ----------------
USERS_FILE = "users.json"

def save_users(users):
    with open(USERS_FILE, "w") as f:
        json.dump(users, f)

def load_users():
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, "r") as f:
            return json.load(f)
    return {}

# ---------------- PREPROCESSING ----------------
lemmatizer = WordNetLemmatizer()
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)

# ---------------- TRAINING DATA ----------------
training_data = [
    ("hello", "greeting"),
    ("hi", "greeting"),
    ("how are you?", "greeting"),
    ("bye", "farewell"),
    ("see you later", "farewell"),
    ("what is your name?", "info"),
    ("tell me about python", "info")
]

corpus = [text for text, label in training_data]
labels = [label for text, label in training_data]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

model = MLPClassifier(hidden_layer_sizes=(8,), max_iter=500, random_state=42)
model.fit(X, labels)

# ---------------- INTENTS ----------------
intents = {
    "intents": [
        {"tag": "greeting", "responses": ["👋 Hello!", "😊 Hi there!", "🙌 Hey!"]},
        {"tag": "farewell", "responses": ["👋 Goodbye!", "👀 See you later!", "✨ Take care!"]},
        {"tag": "info", "responses": ["🤖 I’m a chatbot built with Python.", "💡 I can answer simple questions."]}
    ]
}

# ---------------- INTENT PREDICTION ----------------
def predict_intent(user_input):
    processed = preprocess(user_input)
    X_test = vectorizer.transform([processed])
    probs = model.predict_proba(X_test)[0]
    max_prob = max(probs)
    predicted_label = model.classes_[probs.argmax()]
    if max_prob < 0.6:
        return "unknown"
    return predicted_label

def get_response(intent):
    for i in intents["intents"]:
        if i["tag"] == intent:
            return random.choice(i["responses"])
    return None

# ---------------- GOOGLE SEARCH FALLBACK ----------------
def google_search(query):
    results = []
    for url in search(query):
        results.append(url)
        if len(results) >= 5:
            break
    return results

# ---------------- STREAMLIT UI ----------------
st.set_page_config(page_title="Mini Chatbot", page_icon="🤖", layout="centered")

st.markdown(
    """
    <style>
    .user-msg {
        color: white;
        background-color: #4CAF50;
        padding: 10px;
        border-radius: 8px;
        text-align: right;
        margin: 5px;
    }
    .bot-msg {
        color: black;
        background-color: #e0e0e0;
        padding: 10px;
        border-radius: 8px;
        text-align: left;
        margin: 5px;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# ---------------- SESSION STATE ----------------
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []
if "users" not in st.session_state:
    st.session_state.users = load_users()
if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
if "current_user" not in st.session_state:
    st.session_state.current_user = None

# ---------------- LOGIN / SIGNUP ----------------
if not st.session_state.logged_in:
    st.title("🔐 Login / Signup")
    choice = st.radio("Choose an option:", ["Login", "Signup"])

    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if choice == "Signup":
        if st.button("Create Account"):
            if username in st.session_state.users:
                st.warning("⚠️ Username already exists!")
            else:
                st.session_state.users[username] = password
                save_users(st.session_state.users)
                st.success("✅ Account created! Please login.")
    else:
        if st.button("Login"):
            if username in st.session_state.users and st.session_state.users[username] == password:
                st.session_state.logged_in = True
                st.session_state.current_user = username
                st.success(f"🎉 Welcome back, {username}!")
            else:
                st.error("❌ Invalid username or password.")

# ---------------- CHATBOT ----------------
if st.session_state.logged_in:
    st.title("🤖 Mini Chatbot")
    st.write(f"Logged in as: **{st.session_state.current_user}**")

    temp_input = st.text_input("💬 Type your message here:")

    if st.button("Send"):
        if temp_input:
            intent = predict_intent(temp_input)
            response = get_response(intent)

            if intent == "unknown" or response is None:
                results = google_search(temp_input)
                if results:
                    response = "🔎 I searched the web and found:"
                    st.session_state.chat_history.append(("user", temp_input))
                    st.session_state.chat_history.append(("bot", response))
                    for url in results:
                        st.markdown(f"[🔗 Open Link]({url})")
                    response = None
                else:
                    response = "❓ Sorry, I couldn’t find anything."

            if response:
                st.session_state.chat_history.append(("user", temp_input))
                st.session_state.chat_history.append(("bot", response))

    for sender, msg in st.session_state.chat_history:
        if sender == "user":
            st.markdown(f"<div class='user-msg'>You: {msg}</div>", unsafe_allow_html=True)
        elif msg:
            st.markdown(f"<div class='bot-msg'>Chatbot: {msg}</div>", unsafe_allow_html=True)

    if st.button("Logout"):
        st.session_state.logged_in = False
        st.session_state.current_user = None
        st.session_state.chat_history = []
        st.info("👋 You have been logged out.")

Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🔗 Click this link:", public_url)

🔗 Click this link: NgrokTunnel: "https://homiletic-lakita-uncondensably.ngrok-free.dev" -> "http://localhost:8501"
